In [1]:
import os
import shutil
from tqdm import tqdm

# Patch the shutil.copyfileobj to improve copy speed
def _copyfileobj_patched(fsrc, fdst, length=16*1024*1024):
    """Patches shutil method to hugely improve copy speed."""
    while True:
        buf = fsrc.read(length)
        if not buf:
            break
        fdst.write(buf)

# Replace the original shutil.copyfileobj with the patched one
shutil.copyfileobj = _copyfileobj_patched

def copy_directory_except_group(src, dst, progress_bar):
    """Copy directory except files starting with group."""
    if not os.path.exists(dst):
        os.makedirs(dst)
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            copy_directory_except_group(s, d, progress_bar)
        else:
            if not item.startswith('group'):
                shutil.copy2(s, d)
                progress_bar.update()

def count_files_in_dir(dir_path):
    file_count = 0
    for root, dirs, files in os.walk(dir_path):
        files = [file for file in files if not file.startswith("group")]
        file_count += len(files)
    return file_count

src_folder = 'test.onD/Icaro/raw'
dst_folder = 'test/Icaro/raw'
n_files = count_files_in_dir(src_folder)
progress_bar = tqdm(total=n_files)
copy_directory_except_group(src_folder, dst_folder, progress_bar=progress_bar)


 23%|██▎       | 3172/13539 [06:15<56:29,  3.06it/s]  

OSError: [Errno 28] No space left on device